In [65]:
import numpy as np

import matplotlib
import pandas as pd
import sklearn
from sklearn.datasets import load_digits
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm

np.random.seed(13)


def pprint(x, x_name):
    if len(x.shape) > 1:
        print(f"{x_name} =\n{x}")
    else:
        print(f"{x_name} = {x}")

# 1. Non-negative matrix factorization

## 1.1 Implementation

In [12]:
digits = load_digits()
print(digits.keys())

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])


In [13]:
data = np.array(digits["data"])
images = np.array(digits["images"])
target = np.array(digits["target"])
target_names = digits["target_names"]

In [28]:
pprint(data, "data")
print("data.shape", data.shape)

data =
[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
data.shape (1797, 64)
float64


In [31]:
print("target.shape", target.shape)
print("target.unique =\n", np.stack(np.unique(target, return_counts=True), axis=1))

<class 'numpy.ndarray'>
target.shape (1797,)
target.unique =
 [[  0 178]
 [  1 182]
 [  2 177]
 [  3 183]
 [  4 181]
 [  5 182]
 [  6 181]
 [  7 179]
 [  8 174]
 [  9 180]]


### Seems balanced

In [83]:
def update_H(X, Z_t, H_t):

    num = Z_t.T @ X
    denom = (Z_t.T @ Z_t @ H_t)
    denom = np.clip(denom, a_min=np.nextafter(0, 1), a_max=np.max(denom))

    H_t1 = np.multiply(H_t, (num / denom))

    return H_t1


def update_Z(X, Z_t, H_t1):

    num = X @ H_t1.T
    denom = (Z_t @ H_t1 @ H_t1.T)
    denom = np.clip(denom, a_min=np.nextafter(0,1), a_max=np.max(denom))

    Z_t1 = np.multiply(Z_t, (num / denom))

    return Z_t1


def non_negative(data, num_components):
    """
    Perform non-negative matrix factorization (nmf) on `data`.
    X ≈ Z x H while X,X,H > 0

    :param numpy.ndarray data: Input matrix to apply the mf on.
    :param int num_components: Number of components or features the common dimension will have.
    :return: Z and H --> Two matrices, nmf whose multiplication results approximately in the original data matrix.
    """
    X = data

    N, D = data.shape
    M = num_components

    # Initialize Ht and Zt as H_0 and Z_0
    Z = np.abs(np.random.randn(N, M))
    H = np.abs(np.random.randn(M, D))

    for t in range(1000):

        Z = update_Z(X, Z, H)
        H = update_H(X, Z, H)

    return Z, H


def loss(X, Z, H):
    """
    Frobenius norm
    """

    return np.sum((X - Z @ H)**2)

In [84]:
Z, H = non_negative(data, num_components=10)

743992.2279455089

In [81]:
model = NMF(n_components=10, init="random", max_iter=1000, random_state=13)
Z_skl = model.fit_transform(data)
H_skl = model.components_

## 1.2 Recommender system